In this tutorial, you will learn three approaches to **dealing with missing values**. Then you'll compare the effectiveness of these approaches on a real-world dataset.

# Introduction

There are many ways data can end up with missing values. For example,
- A 2 bedroom house won't include a value for the size of a third bedroom.
- A survey respondent may choose not to share his income.

Most machine learning libraries (including scikit-learn) give an error if you try to build a model using data with missing values. So you'll need to choose one of the strategies below.

# Three Approaches


### 1) A Simple Option: Drop Columns with Missing Values

The simplest option is to drop columns with missing values. 

![tut2_approach1](https://i.imgur.com/Sax80za.png)

Unless most values in the dropped columns are missing, the model loses access to a lot of (potentially useful!) information with this approach.  As an extreme example, consider a dataset with 10,000 rows, where one important column is missing a single entry. This approach would drop the column entirely!

### 2) A Better Option: Imputation

**Imputation** fills in the missing values with some number.  For instance, we can fill in the mean value along each column. 

![tut2_approach2](https://i.imgur.com/4BpnlPA.png)

The imputed value won't be exactly right in most cases, but it usually leads to more accurate models than you would get from dropping the column entirely.

### 3) An Extension To Imputation

Imputation is the standard approach, and it usually works well. However, imputed values may be systematically above or below their actual values (which weren't collected in the dataset). Or rows with missing values may be unique in some other way. In that case, your model would make better predictions by considering which values were originally missing.  

![tut3_approach3](https://i.imgur.com/UWOyg4a.png)

In this approach, we impute the missing values, as before.  And, additionally, for each column with missing entries in the original dataset, we add a new column that shows the location of the imputed entries.

In some cases, this will meaningfully improve results. In other cases, it doesn't help at all.

# Example 

In the example, we will work with the [Melbourne Housing dataset](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot/home).  Our model will use information such as the number of rooms and land size to predict home price.

We won't focus on the data loading step. Instead, you can imagine you are at a point where you already have the training and validation data in `X_train`, `X_valid`, `y_train`, and `y_valid`. 

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')

print(data.columns)
# Select target
y = data.Price # column that we need to predict

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')


In [23]:
melb_predictors = data.drop(['Price'], axis=1)
print(melb_predictors.columns) # see price column is gone now
melb_predictors.shape

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Method', 'SellerG', 'Date',
       'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize',
       'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude', 'Longtitude',
       'Regionname', 'Propertycount'],
      dtype='object')


(13580, 20)

In [29]:
for col in data.columns:
    print(col ,"-->",data[col].dtype)
# columns with strings are marked as objects

Suburb --> object
Address --> object
Rooms --> int64
Type --> object
Price --> float64
Method --> object
SellerG --> object
Date --> object
Distance --> float64
Postcode --> float64
Bedroom2 --> float64
Bathroom --> float64
Car --> float64
Landsize --> float64
BuildingArea --> float64
YearBuilt --> float64
CouncilArea --> object
Lattitude --> float64
Longtitude --> float64
Regionname --> object
Propertycount --> float64


In [47]:
# To keep things simple, we'll use only numerical predictors
X = melb_predictors.select_dtypes(exclude=['object']) # take all the dtypes except strings

print(X.columns)
print(X.shape)
print(X["BuildingArea"].describe()) # only 7130 values have non nan in this column
# hence lot's of missing values

Index(['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude',
       'Propertycount'],
      dtype='object')
(13580, 12)
count     7130.000000
mean       151.967650
std        541.014538
min          0.000000
25%         93.000000
50%        126.000000
75%        174.000000
max      44515.000000
Name: BuildingArea, dtype: float64


In [51]:
# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=
                                                      0.2,random_state=0)
# X are DataFrames while y are series

### Define Function to Measure Quality of Each Approach

We define a function `score_dataset()` to compare different approaches to dealing with missing values. This function reports the [mean absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error) (MAE) from a random forest model.

In [52]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Score from Approach 1 (Drop Columns with Missing Values)

Since we are working with both training and validation sets, we are careful to drop the same columns in both DataFrames.  

In [64]:
print(X_train["BuildingArea"].isnull())
print("Number of missing values = ",sum(X_train["BuildingArea"].isnull()))
print("Any missing value ?",X_train["Rooms"].isnull().any()  ) 

12167     True
6524      True
8413      True
2919      True
6043     False
         ...  
13123     True
3264     False
9845     False
10799     True
2732     False
Name: BuildingArea, Length: 10864, dtype: bool
Number of missing values =  5156
Any missing value ? False


In [73]:
# Get names of COLUMNS with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
print(X_train.shape,reduced_X_train.shape)

reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)
print(X_valid.shape,reduced_X_valid.shape)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

(10864, 12) (10864, 9)
(2716, 12) (2716, 9)
MAE from Approach 1 (Drop columns with missing values):
183550.22137772635


### Score from Approach 2 (Imputation)

Next, we use [`SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) to replace missing values with the mean value along each column.

Although it's simple, filling in the mean value generally performs quite well (but this varies by dataset).  While statisticians have experimented with more complex ways to determine imputed values (such as **regression imputation**, for instance), the complex strategies typically give no additional benefit once you plug the results into sophisticated machine learning models.

In [104]:
import numpy as np
from sklearn.impute import SimpleImputer 
imp = SimpleImputer(strategy = "mean",missing_values = np.nan)

train_data = np.array([[1, 2], [np.nan, 3], [7, 6]] )
print("train_data\n",train_data)


# the Imputer has to calculate the mean first.
imp.fit(train_data) 
# This step of calculating that value is called the fit() method.


train_data
 [[ 1.  2.]
 [nan  3.]
 [ 7.  6.]]


SimpleImputer()


Next, the transform() method will just replace the NaNs in the column with 
the newly calculated value, and return the new dataset.


In [105]:
print("Data After Transformation is \n",imp.transform(train_data))

Data After Transformation is 
 [[1. 2.]
 [4. 3.]
 [7. 6.]]


The fit_transform() method will do both the things internally and makes it easy for us by 
just exposing one single method. But there are instances where you want to call only the 
fit() method and only the transform() method.


When you are training a model, you will use the training dataset. On this dataset,
you’ll use the Imputer, calculate the value, and replace the blanks. 
But when you fit this trained model on the test dataset, you don’t calculate the mean or
median again. You’ll use the same value that you used on your training dataset. 
For this, you’ll use the fit() method on your training dataset to only calculate the 
value and keep it internally in the Imputer. Then, you’ll call the transform() method on
the test dataset with the same Inputer object. This way, the value calculate for the 
training set, which was saved internally in the object, will be used on the test dataset as
well.


In [106]:
test_data = np.array([[np.nan, 2], [6, np.nan], [7, 6]])
print("test_data\n",test_data)

print("test Data after transformation is \n",imp.transform(test_data))
# mean of first column is not 4 according to test data but we are filling it wrt to train data

test_data
 [[nan  2.]
 [ 6. nan]
 [ 7.  6.]]
test Data after transformation is 
 [[4.         2.        ]
 [6.         3.66666667]
 [7.         6.        ]]


To put it simply, you can use the fit_transform() method on the training set, as you’ll need to both fit and transform the data, and you can use the fit() method on the training dataset to get the value, and later transform() test data with it. 

In [136]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
print("Type of training-data is ",type(X_train))

my_imputer.fit(X_train)
imputed_X_train = my_imputer.transform(X_train) # it will return Numpy array instead of
print("Type of returned Data is ",type(imputed_X_train))
# but we want our data of type DataFrame instead of numpy array

Type of training-data is  <class 'pandas.core.frame.DataFrame'>
Type of returned Data is  <class 'numpy.ndarray'>


In [138]:
imputed_X_train = pd.DataFrame(my_imputer.transform(X_train))
print("type--",type(imputed_X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

type-- <class 'pandas.core.frame.DataFrame'>


In [140]:
 from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

print(imputed_X_train.columns)
# returned data is of type DataFrame now but it's column names are changed
print(imputed_X_train.head())
# initially X_train was a DataFrame but imputer changed it's column names

RangeIndex(start=0, stop=12, step=1)
    0     1       2    3    4    5      6           7            8         9   \
0  1.0   5.0  3182.0  1.0  1.0  1.0    0.0  153.764119  1940.000000 -37.85984   
1  2.0   8.0  3016.0  2.0  2.0  1.0  193.0  153.764119  1964.839866 -37.85800   
2  3.0  12.6  3020.0  3.0  1.0  1.0  555.0  153.764119  1964.839866 -37.79880   
3  3.0  13.0  3046.0  3.0  1.0  1.0  265.0  153.764119  1995.000000 -37.70830   
4  3.0  13.3  3020.0  3.0  1.0  2.0  673.0  673.000000  1970.000000 -37.76230   

         10       11  
0  144.9867  13240.0  
1  144.9005   6380.0  
2  144.8220   3755.0  
3  144.9158   8870.0  
4  144.8272   4217.0  


In [144]:
# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print(imputed_X_train.columns)

Index(['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude',
       'Propertycount'],
      dtype='object')


In [145]:
print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
178166.46269899711


We see that **Approach 2** has lower MAE than **Approach 1**, so **Approach 2** performed better on this dataset.

### Score from Approach 3 (An Extension to Imputation)

Next, we impute the missing values, while also keeping track of which values were imputed.

In [187]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

print(X_train.columns)
print("\n\n X_train \n",X_train.iloc[2:8,7:])

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    # remeber isnull() return true ifor all the cells where cells are nan.
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()
    
print("\n\n",X_train_plus.columns)
print("\n\n X_train_plus\n\n",X_train_plus.iloc[2:8,-3:])

Index(['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude',
       'Propertycount'],
      dtype='object')


 X_train 
       BuildingArea  YearBuilt  Lattitude  Longtitude  Propertycount
8413           NaN        NaN   -37.7988    144.8220         3755.0
2919           NaN     1995.0   -37.7083    144.9158         8870.0
6043         673.0     1970.0   -37.7623    144.8272         4217.0
547            NaN        NaN   -37.8116    145.0789         5682.0
4655         194.0     1983.0   -37.7319    144.9461         7485.0
6082           NaN        NaN   -37.7845    144.8131         6763.0


 Index(['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude',
       'Propertycount', 'Car_was_missing', 'BuildingArea_was_missing',
       'YearBuilt_was_missing'],
      dtype='object')


 X_train_plus

       Car_was_missing  Bu

In [188]:
# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


As we can see, **Approach 3** performed slightly worse than **Approach 2**.

### So, why did imputation perform better than dropping the columns?

The training data has 10864 rows and 12 columns, where three columns contain missing data.  For each column, less than half of the entries are missing.  Thus, dropping the columns removes a lot of useful information, and so it makes sense that imputation would perform better.

In [192]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


# Conclusion
As is common, imputing missing values (in **Approach 2** and **Approach 3**) yielded better results, relative to when we simply dropped columns with missing values (in **Approach 1**).

# Your Turn

Compare these approaches to dealing with missing values yourself in **[this exercise](https://www.kaggle.com/kernels/fork/3370280)**!

---




*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161289) to chat with other Learners.*